In [ ]:
#Create a cluster
import ipyparallel as ipp

# Create an MPI cluster with 8 engines
nproc = 8 #<-- Para usar as 8 GPUS da DGX
cluster = ipp.Cluster(engines="mpi", n=nproc)

# Start and connect to the cluster
rc = cluster.start_and_connect_sync()

# Enable IPython magics for parallel processing
rc[:].activate()

A partir daqui use a mágica ``%%px --block`` para executar no cluster criado

In [ ]:
%%px --block

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import rockverse as rv
from rockverse.optimize import gaussian_val

samples = pd.read_csv('/nethome/drp2/servicos/ROCHA_DIGITAL/CERD_UFRJ/Rodolfo/rodolfo.csv')

######################################
# Input parameters
specs = {
    #'sample': 'V017973H',
    'resolution': 41740,
    'bhc': True,
    'acquisition': 'long',
}
energies = (160, 220)

group_name = 'Test1' # Directory name

overwrite = True # Overwite directory by default?

workdir = '/nethome/drp2/servicos/ROCHA_DIGITAL/CERD_UFRJ/Rodolfo/'
#####################################

#Get sample paths
sample_set = samples
for k, v in specs.items():
    sample_set = sample_set[sample_set[k] == v]

lowE_sample_info = sample_set[sample_set['energy']==min(energies)]
highE_sample_info = sample_set[sample_set['energy']==max(energies)]


# Cria o grupo e carrega as imagens

In [ ]:
%%px --block

dectgroup = rv.open(store=os.path.join(workdir, group_name))


# Create the Dual Energy CT group
dectgroup = rv.dect.create_group(
    store=os.path.join(workdir, group_name),
    overwrite=overwrite)

#Parâmetros poderiam estar naquele tabelão...
shape = (1404, 1412, 1266)
dtype = '<i2' # little-endian 16bit signed integer
chunks = (200, 200, 200)
voxel_length = (1, 1, 1)


# Import the low energy image
dectgroup.copy_image(
    name='lowECT',
    image=rv.voxel_image.import_raw(
        rawfile=os.path.join(workdir,  lowE_sample_info['path'].iloc[0].split(os.sep)[-1]) + '_import.raw',
        shape=shape,                         #<- From metadata, image size
        chunks=chunks,                       #<- Our choice of chunk size will give a 2x2x2 chunk grid
        dtype=dtype,                         #<- From metadata, little-endian 16-bit signed integer
        offset=0,                            #<- From metadata
        voxel_length=voxel_length,
        voxel_unit='u.a.',                   #<- From metadata
        raw_file_order='F',                  #<- Fortran file order
        field_name='Low energy attenuation', #<- Our choice for field name (X-ray attenuation)
        field_unit='',                       #<- field units
        description='Low energy X-ray CT attenuation',
        overwrite=True                       #<- Overwrite if file exists in disk
        )
    )

# Import the high energy image
dectgroup.copy_image(
    name='highECT',
    image=rv.voxel_image.import_raw(
        rawfile=os.path.join(workdir,  highE_sample_info['path'].iloc[0].split(os.sep)[-1]) + '_import.raw',
        shape=shape,                         #<- From metadata, image size
        chunks=chunks,                       #<- Our choice of chunk size will give a 2x2x2 chunk grid
        dtype=dtype,                         #<- From metadata, little-endian 16-bit signed integer
        offset=0,                            #<- From metadata
        voxel_length=voxel_length,
        voxel_unit='u.a.',                   #<- From metadata
        raw_file_order='F',                  #<- Fortran file order
        field_name='High energy attenuation', #<- Our choice for field name (X-ray attenuation)
        field_unit='',                       #<- field units
        description='High energy X-ray CT attenuation',
        overwrite=True                       #<- Overwrite if file exists in disk
        )
    )

# Optional: Import segmentation
dectgroup.copy_image(
    name='segmentation',
    image=rv.voxel_image.import_raw(
        rawfile=os.path.join(workdir,  lowE_sample_info['path'].iloc[0].split(os.sep)[-1]) + '_seed.raw',
        shape=shape,                         #<- From metadata, image size
        chunks=chunks,                       #<- Our choice of chunk size will give a 2x2x2 chunk grid
        dtype='|u1',                          #<- 8-bit unsigned integer
        offset=0,                            #<- From metadata
        voxel_length=voxel_length,
        voxel_unit='u.a.',                   #<- From metadata
        raw_file_order='F',                  #<- Fortran file order
        field_name='Segmentation',
        field_unit='',                       #<- field units
        description='Segmentation',
        overwrite=True                       #<- Overwrite if file exists in disk
        )
    )

# mask out empty space
dectgroup.create_mask(fill_value=False, overwrite=True)
dectgroup.mask.math(value=True,
                    op='set',
                    segmentation=dectgroup.segmentation,
                    phases=(0,))


# Opcional: Visualização das imagens

In [ ]:
%%px --block

lowE_viewer = rv.OrthogonalViewer(image=dectgroup.lowECT)
highE_viewer = rv.OrthogonalViewer(image=dectgroup.lowECT)

if rv.mpi_rank != 0:
    plt.close(lowE_viewer.figure)
    plt.close(highE_viewer.figure)

# Coloca as info dos calibradores

In [ ]:
%%px --block

### Ar ---------------------------------------------------
ind = 0
dectgroup.calibration_material[ind].description = 'Ar'

m = 0
s = 0.01
dectgroup.calibration_material[ind].lowE_gaussian_pdf = (m, s)

m = 0
s = 0.01
dectgroup.calibration_material[ind].highE_gaussian_pdf = (m, s)


### Alumínio --------------------------------------------------
ind = 1
dectgroup.calibration_material[ind].description = 'Alumínio'
dectgroup.calibration_material[ind].bulk_density = 2.7
dectgroup.calibration_material[ind].composition = {'Al': 1}

m = lowE_sample_info['mean_aluminio'].iloc[0]
s = lowE_sample_info['desv_aluminio'].iloc[0]
dectgroup.calibration_material[ind].lowE_gaussian_pdf = (m, s)

m = highE_sample_info['mean_aluminio'].iloc[0]
s = highE_sample_info['desv_aluminio'].iloc[0]
dectgroup.calibration_material[ind].highE_gaussian_pdf = (m, s)



### Teflon --------------------------------------------------
ind = 2
dectgroup.calibration_material[ind].description = 'Teflon'
dectgroup.calibration_material[ind].bulk_density = 2.2
dectgroup.calibration_material[ind].composition = {'C': 2, 'F': 4}

m = lowE_sample_info['mean_teflon'].iloc[0]
s = lowE_sample_info['desv_teflon'].iloc[0]
dectgroup.calibration_material[ind].lowE_gaussian_pdf = (m, s)

m = highE_sample_info['mean_teflon'].iloc[0]
s = highE_sample_info['desv_teflon'].iloc[0]
dectgroup.calibration_material[ind].highE_gaussian_pdf = (m, s)



### Sílica --------------------------------------------------
ind = 3
dectgroup.calibration_material[ind].description = 'SiO2'
dectgroup.calibration_material[ind].bulk_density = 2.2
dectgroup.calibration_material[ind].composition = {'Si': 1, 'O': 2}

m = lowE_sample_info['mean_quartzo'].iloc[0]
s = lowE_sample_info['desv_quartzo'].iloc[0]
dectgroup.calibration_material[ind].lowE_gaussian_pdf = (m, s)

m = highE_sample_info['mean_quartzo'].iloc[0]
s = highE_sample_info['desv_quartzo'].iloc[0]
dectgroup.calibration_material[ind].highE_gaussian_pdf = (m, s)

# Executando a inversão nesse mini cluster local

In [ ]:
%%px --block
dectgroup.run(restart=True)

# Se quiser jogar no cluster

Faça as configurações iniciais acima e use esses scripts

## Bash script:

```bash
#!/bin/bash
#SBATCH -J awesome_job_name
#SBATCH --nodes=2
#SBATCH --partition <your_partition>
#SBATCH --gres=gpu:16
#SBATCH --ntasks-per-node=256
#SBATCH --output=outputlog
#SBATCH --chdir=/path/to/workdir

conda activate you_environment_with_rockverse1.0
mpirun python3 ./run.py 
```

## Python script

```python
import rockverse as rv
dectgroup = rv.open(r'/path/to/dual_energy_ct/group')
dectgroup.run()
```